In [1]:
# Freight Class Breakpoints
class_breakpoints = [
    ("L5C", 0, 499),
    ("5C", 500, 999),
    ("1M", 1000, 1999),
    ("2M", 2000, 2999),
    ("3M", 3000, 4999),
    ("5M", 5000, 9999),
    ("10M", 10000, 19999),
    ("20M", 20000, 29999),
    ("30M", 30000, 39999),
    ("40M", 40000, float("inf")),
]

# Freight Rates
rates = {
    "SQYD": {
        "Carpet": {
            "L5C": 0.4366, "5C": 0.4249, "1M": 0.419, "2M": 0.4102, "3M": 0.3955
        },
        "Carpet Tile": {
            "L5C": 0.7422, "5C": 0.7223, "1M": 0.7123, "2M": 0.6974, "3M": 0.6724
        }
    },
    "CWT": {
        60: {  # class 60
            "L5C": 20.57, "5C": 18.74, "1M": 14.53, "2M": 11.69, "3M": 11.69,
            "5M": 9.07, "10M": 7.64, "20M": 7.64, "30M": 7.64, "40M": 7.64
        },
        70: {
            "L5C": 22.87, "5C": 20.85, "1M": 16.16, "2M": 13.00, "3M": 13.00,
            "5M": 10.19, "10M": 8.49, "20M": 8.49, "30M": 8.49, "40M": 8.49
        },
        150: {
            "L5C": 45.29, "5C": 41.28, "1M": 232.01, "2M": 25.74, "3M": 25.74,
            "5M": 20.17, "10M": 14.95, "20M": 14.95, "30M": 14.95, "40M": 14.95
        }
    }
}

# Discounts
discounts = {
    "SQYD": 0.7885,
    "CWT": 0.7795
}

# Convert SQFT to SQYD (1 SQYD = 9 SQFT)
def sqft_to_sqyd(sqft):
    return sqft / 9

# Determine freight class based on quantity
def get_priority_class(quantity: float) -> str:
    for class_name, min_q, max_q in class_breakpoints:
        if min_q <= quantity <= max_q:
            return class_name
    raise ValueError("Quantity is out of range.")

# Main estimator function
def estimate_freight_cost(uom: str, quantity: float, product_type: str = None, cwt_class: int = None) -> float:
    uom = uom.upper()

    # Convert if needed
    if uom == "SQFT":
        quantity = sqft_to_sqyd(quantity)
        uom = "SQYD"

    # Determine freight class
    priority_class = get_priority_class(quantity)
    print(f"Determined priority class: {priority_class}")

    # Calculate cost
    if uom == "SQYD":
        if product_type not in rates["SQYD"]:
            raise ValueError(f"Unsupported product type '{product_type}' for SQYD.")
        rate = rates["SQYD"][product_type].get(priority_class)
        if rate is None:
            raise ValueError(f"Rate for class {priority_class} not available for {product_type}.")
        discount = discounts["SQYD"]
    elif uom == "CWT":
        if cwt_class not in rates["CWT"]:
            raise ValueError(f"Unsupported CWT class {cwt_class}.")
        rate = rates["CWT"][cwt_class].get(priority_class)
        if rate is None:
            raise ValueError(f"Rate for CWT class {cwt_class} and priority class {priority_class} not found.")
        discount = discounts["CWT"]
    else:
        raise ValueError(f"Unsupported unit of measure '{uom}'.")

    final_rate = rate * discount
    total_cost = final_rate * quantity
    return round(total_cost, 2)

# --- Example usage 1: SQFT product ---
print("Example 1: Carpet Tile in SQFT")
print(estimate_freight_cost(uom="SQFT", quantity=9000, product_type="Carpet Tile"))  # auto-converted to SQYD

# --- Example usage 2: CWT product ---
print("\nExample 2: CWT product, class 70")
print(estimate_freight_cost(uom="CWT", quantity=6000, cwt_class=70))


Example 1: Carpet Tile in SQFT
Determined priority class: 1M
561.65

Example 2: CWT product, class 70
Determined priority class: 5M
47658.63


In [ ]:
# --- Freight Class Breakpoints ---
class_breakpoints = [
    ("L5C", 0, 499),
    ("5C", 500, 999),
    ("1M", 1000, 1999),
    ("2M", 2000, 2999),
    ("3M", 3000, 4999),
    ("5M", 5000, 9999),
    ("10M", 10000, 19999),
    ("20M", 20000, 29999),
    ("30M", 30000, 39999),
    ("40M", 40000, float("inf")),
]

# --- Location-Based Rates ---
rates = {
    "SQYD": {
        "Texas": {
            "Carpet": {
                "L5C": 0.45, "5C": 0.42, "1M": 0.41
            },
            "Carpet Tile": {
                "L5C": 0.75, "5C": 0.72, "1M": 0.70
            }
        },
        "Florida": {
            "Carpet": {
                "L5C": 0.50, "5C": 0.47, "1M": 0.45
            },
            "Carpet Tile": {
                "L5C": 0.78, "5C": 0.74, "1M": 0.71
            }
        }
    },
    "CWT": {
        "Texas": {
            60: {
                "L5C": 20.0, "5C": 18.5, "1M": 14.2
            },
            70: {
                "L5C": 23.0, "5C": 21.0, "1M": 16.5
            }
        },
        "Florida": {
            60: {
                "L5C": 21.0, "5C": 19.0, "1M": 15.0
            },
            70: {
                "L5C": 24.0, "5C": 22.0, "1M": 17.0
            }
        }
    }
}

# --- Location-Based Discounts ---
discounts = {
    "SQYD": {
        "Texas": 0.78,
        "Florida": 0.76
    },
    "CWT": {
        "Texas": 0.77,
        "Florida": 0.74
    }
}

# --- Convert SQFT to SQYD ---
def sqft_to_sqyd(sqft):
    return sqft / 9

# --- Get Priority Class from Quantity ---
def get_priority_class(quantity: float) -> str:
    for class_name, min_q, max_q in class_breakpoints:
        if min_q <= quantity <= max_q:
            return class_name
    raise ValueError("Quantity is out of range.")

# --- Main Estimation Function ---
def estimate_freight_cost(uom: str, quantity: float, location: str,
                          product_type: str = None, cwt_class: int = None) -> float:
    uom = uom.upper()
    location = location.title()

    if uom == "SQFT":
        quantity = sqft_to_sqyd(quantity)
        uom = "SQYD"

    # Determine class
    priority_class = get_priority_class(quantity)
    print(f"Determined freight class: {priority_class}")

    # Look up rate and discount
    if uom == "SQYD":
        try:
            rate = rates["SQYD"][location][product_type][priority_class]
            discount = discounts["SQYD"][location]
        except KeyError:
            raise ValueError(f"Missing rate or discount for {product_type} in {location} ({priority_class})")
    elif uom == "CWT":
        try:
            rate = rates["CWT"][location][cwt_class][priority_class]
            discount = discounts["CWT"][location]
        except KeyError:
            raise ValueError(f"Missing CWT rate or discount for class {cwt_class} in {location} ({priority_class})")
    else:
        raise ValueError(f"Unsupported unit of measure '{uom}'")

    final_rate = rate * discount
    total_cost = round(final_rate * quantity, 2)
    return total_cost

# --- Test Cases ---
print("Example 1: Carpet Tile, 9000 SQFT, Texas")
print("Estimated Freight Cost: $", estimate_freight_cost(
    uom="SQFT", quantity=9000, location="Texas", product_type="Carpet Tile"))

print("\nExample 2: CWT, class 70, 7000 lbs, Florida")
print("Estimated Freight Cost: $", estimate_freight_cost(
    uom="CWT", quantity=7000, location="Florida", cwt_class=70))


In [ ]:
import pandas as pd

# --------------------------
# Freight Class Breakpoints
# --------------------------
class_breakpoints = [
    ("L5C", 0, 499),
    ("5C", 500, 999),
    ("1M", 1000, 1999),
    ("2M", 2000, 2999),
    ("3M", 3000, 4999),
    ("5M", 5000, 9999),
    ("10M", 10000, 19999),
    ("20M", 20000, 29999),
    ("30M", 30000, 39999),
    ("40M", 40000, float("inf")),
]

# --------------------------
# Location-based Rates
# --------------------------
rates = {
    "SQYD": {
        "Texas": {
            "Carpet": {"L5C": 0.45, "5C": 0.42, "1M": 0.41},
            "Carpet Tile": {"L5C": 0.75, "5C": 0.72, "1M": 0.70}
        },
        "Florida": {
            "Carpet": {"L5C": 0.50, "5C": 0.47, "1M": 0.45},
            "Carpet Tile": {"L5C": 0.78, "5C": 0.74, "1M": 0.71}
        }
    },
    "CWT": {
        "Texas": {
            60: {"L5C": 20.0, "5C": 18.5, "1M": 14.2},
            70: {"L5C": 23.0, "5C": 21.0, "1M": 16.5}
        },
        "Florida": {
            60: {"L5C": 21.0, "5C": 19.0, "1M": 15.0},
            70: {"L5C": 24.0, "5C": 22.0, "1M": 17.0}
        }
    }
}

# --------------------------
# Location-based Discounts
# --------------------------
discounts = {
    "SQYD": {
        "Texas": 0.78,
        "Florida": 0.76
    },
    "CWT": {
        "Texas": 0.77,
        "Florida": 0.74
    }
}

# --------------------------
# Helpers
# --------------------------
def sqft_to_sqyd(sqft):
    return sqft / 9

def get_priority_class(quantity: float) -> str:
    for class_name, min_q, max_q in class_breakpoints:
        if min_q <= quantity <= max_q:
            return class_name
    raise ValueError("Quantity is out of range.")

# --------------------------
# Main Freight Estimator
# --------------------------
def estimate_freight_cost(uom: str, quantity: float, location: str,
                          product_type: str = None, cwt_class: int = None) -> float:
    uom = uom.upper()
    location = location.title()

    if uom == "SQFT":
        quantity = sqft_to_sqyd(quantity)
        uom = "SQYD"

    priority_class = get_priority_class(quantity)

    if uom == "SQYD":
        try:
            rate = rates["SQYD"][location][product_type][priority_class]
            discount = discounts["SQYD"][location]
        except KeyError:
            raise ValueError(f"Missing rate or discount for {product_type} in {location} ({priority_class})")
    elif uom == "CWT":
        try:
            rate = rates["CWT"][location][cwt_class][priority_class]
            discount = discounts["CWT"][location]
        except KeyError:
            raise ValueError(f"Missing CWT rate or discount for class {cwt_class} in {location} ({priority_class})")
    else:
        raise ValueError(f"Unsupported unit of measure '{uom}'")

    return round(rate * discount * quantity, 2)

# --------------------------
# Example Batch Processor from Excel-like DataFrame
# --------------------------
def apply_freight_estimation(df: pd.DataFrame) -> pd.DataFrame:
    results = []

    for _, row in df.iterrows():
        try:
            result = estimate_freight_cost(
                uom=row["PURCH UOM"],
                quantity=row["PO PURCH QTY"],
                location=row["SHIP TO ZIP"],  # you may want to map ZIP to state
                product_type=row.get("PART DESCRIPTION"),
                cwt_class=row.get("Commodity Group")  # or another column if it's CWT
            )
        except Exception as e:
            result = f"Error: {e}"
        results.append(result)

    df["ESTIMATED_FREIGHT_COST"] = results
    return df

# Import everything from the freight estimator script (assume it's defined above this block)

# ✅ Step 1: Create a sample input DataFrame
data = [
    {
        "PURCH UOM": "SQFT",
        "PO PURCH QTY": 9000,
        "SHIP TO ZIP": "Texas",
        "PART DESCRIPTION": "Carpet Tile",
        "Commodity Group": 70
    },
    {
        "PURCH UOM": "SQYD",
        "PO PURCH QTY": 1000,
        "SHIP TO ZIP": "Florida",
        "PART DESCRIPTION": "Carpet",
        "Commodity Group": 60
    },
    {
        "PURCH UOM": "CWT",
        "PO PURCH QTY": 7000,
        "SHIP TO ZIP": "Texas",
        "PART DESCRIPTION": None,
        "Commodity Group": 60
    }
]

df = pd.DataFrame(data)

# ✅ Step 2: Apply the freight estimation to each row
output_df = apply_freight_estimation(df)

# ✅ Step 3: Display the result
print(output_df)
